<a href="https://colab.research.google.com/github/bilalProgTech/btech-nmims/blob/master/NN-DL/Lab-Work/20230929-Lab-8-2-BTech-NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign

 94% 305M/325M [00:01<00:00, 185MB/s]
100% 325M/325M [00:01<00:00, 214MB/s]


In [ ]:
!unzip skin-cancer-malignant-vs-benign.zip

In [ ]:
!pip install fastai==1.0.61

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

from IPython.display import Image, display

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [5]:
from fastai.imports import *
from fastai import *
from fastai.vision import *

In [6]:
tfms = get_transforms(do_flip = True,
                      max_rotate = 40)

data = ImageDataBunch.from_folder('/content/train',
                                  valid_pct = 0.2,
                                  bs = 32,
                                  size = 256).normalize(imagenet_stats)

In [ ]:
learn_resnet101 = cnn_learner(data, models.resnet101, metrics=accuracy)
learn_resnet101.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 143MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.597524,0.331022,0.848197,00:43
1,0.375145,0.349595,0.838710,00:30


In [ ]:
learn_resnet50 = cnn_learner(data, models.resnet50, metrics=accuracy)
learn_resnet50.fit(epochs = 10)

In [ ]:
learn_resnet152 = cnn_learner(data, models.resnet152, metrics=accuracy)
learn_resnet152.fit(epochs = 10)

In [ ]:
learn_densenet161 = cnn_learner(data, models.densenet161, metrics=accuracy)
learn_densenet161.fit(epochs = 10)

In [ ]:
img = open_image('/content/dataset/single_prediction/cat_or_dog_1.jpg')
learn_densenet161.predict(img)[0]

In [ ]:
img = open_image('/content/dataset/single_prediction/cat_or_dog_2.jpg')
learn_densenet161.predict(img)[0]

In [ ]:
files = []
target = []
for dirname, _, filenames in os.walk('/content/test'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        files.append(filepath)
        target.append(filepath.split('/')[-2])
test = pd.DataFrame({'files': files, 'target': target})
test = test.sample(frac=1)
test = test.reset_index(drop=True)
test.head()

In [ ]:
def predict_cat_or_dog(model_learn, filepath):
    img = open_image(filepath)
    return model_learn.predict(img)[0]

In [ ]:
test['resnet50'] = test['files'].apply(lambda x: predict_cat_or_dog(learn_resnet50, x))
test['resnet50'] = test['resnet50'].astype('str')

In [ ]:
test.head()

In [ ]:
test['densenet161'] = test['files'].apply(lambda x: predict_cat_or_dog(learn_densenet161, x))
test['densenet161'] = test['densenet161'].astype('str')
test.head()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(test['target'], test['resnet50'])

In [ ]:
confusion_matrix(test['target'], test['resnet50'])

In [ ]:
accuracy_score(test['target'], test['densenet161'])

In [ ]:
confusion_matrix(test['target'], test['densenet161'])

In [ ]:
learn_vgg16 = cnn_learner(data, models.vgg16_bn, metrics=accuracy)
learn_vgg16.fit(epochs = 10)

In [ ]:
test['vgg16'] = test['files'].apply(lambda x: predict_cat_or_dog(learn_vgg16, x))
test['vgg16'] = test['vgg16'].astype('str')
test.head()

In [ ]:
accuracy_score(test['target'], test['vgg16'])

In [ ]:
confusion_matrix(test['target'], test['vgg16'])